# Bert Regressor

In [1]:
import tensorflow as tf
import modeling
import tokenization

import run_regressor

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
bert_config_file = '/home/ubuntu/bert_models/uncased_L-12_H-768_A-12/bert_config.json'
bert_config = modeling.BertConfig.from_json_file(bert_config_file)
vocab_file = '/home/ubuntu/bert_models/uncased_L-12_H-768_A-12/vocab.txt'

In [3]:
max_seq_length = 128

In [4]:
processor = run_regressor.ArdProcessor()

In [5]:
tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file,do_lower_case=True)

In [6]:
test_examples = processor.get_test_examples('/home/ubuntu/glue_data/ARD/')

In [7]:
test_features = run_regressor.convert_examples_to_features(test_examples,
                                                            max_seq_length=max_seq_length,
                                                            tokenizer=tokenizer)

INFO:tensorflow:Writing example 0 of 3191
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-0
INFO:tensorflow:tokens: [CLS] i purchased this tablet out of enthusiasm for it ' s built in beats audio . while it does have beats audio built in , it is a very limited feature . the overall movement through the tablet is very smooth . however , i noticed early on that what should be beautiful hd graphics were grain ##y and down right ugly . i usually get nice photos on my facebook wall . on my hp laptop , and my samsung tablet the photos look beautiful , but on the slate 7 the pictures are terrible . that was my first concern . second concern , the slate 7 didn ' t come with a built in video player . the primary reason i started using [SEP]
INFO:tensorflow:input_ids: 101 1045 4156 2023 13855 2041 1997 12024 2005 2009 1005 1055 2328 1999 10299 5746 1012 2096 2009 2515 2031 10299 5746 2328 1999 1010 2009 2003 1037 2200 3132 3444 1012 1996 3452 2929 2083 1996 13855 2003 2200 5744 1012 2

INFO:tensorflow:input_ids: 101 2009 2453 2022 1996 3453 1012 1045 2031 7791 2000 1037 2193 1997 7492 1998 3432 1045 2572 2467 2921 5782 2062 1012 1998 2011 2062 1010 1045 2812 2613 2614 2008 2003 3019 1998 2583 2000 6039 1037 2282 2007 7496 1012 2005 1996 1002 2531 4696 1011 1011 2023 2003 2011 2521 1996 2190 2518 1045 2031 7791 2000 1012 2348 2025 3819 1010 2009 2003 2428 2204 1012 2054 2087 2614 2066 2003 2019 2214 12689 27291 2557 2008 2085 5366 1002 3263 1012 1012 1012 1012 1998 2008 2003 5621 2129 1045 2514 2027 2614 1012 2045 2003 1037 2843 1997 1044 18863 2007 21299 1010 7209 5551 1010 9389 8758 1010 14595 2243 1010 1046 16558 1011 1011 1011 2017 2171 102
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 

In [8]:
all_input_ids = []
all_input_mask = []
all_segment_ids = []
all_label_ids = []

In [9]:
for feature in test_features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)


In [10]:
input_ids=tf.constant(all_input_ids[:50])
input_mask=tf.constant(all_input_mask[:50])
segment_ids=tf.constant(all_segment_ids[:50])
label_ids = tf.constant(all_label_ids[:50])

In [11]:
model = modeling.BertModel(
  config=bert_config,
  is_training=True,
  input_ids=input_ids,
  input_mask=input_mask,
  token_type_ids=segment_ids,
  use_one_hot_embeddings=False)

In [12]:
num_labels = 1
is_training = True

In [13]:
output_layer = model.get_pooled_output()

hidden_size = output_layer.shape[-1].value

output_weights = tf.get_variable(
  "output_weights", [1, hidden_size],
  initializer=tf.truncated_normal_initializer(stddev=0.02))

output_bias = tf.get_variable(
  "output_bias", [num_labels], initializer=tf.zeros_initializer())

with tf.variable_scope("loss"):
    if is_training:
        # I.e., 0.1 dropout
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    #probabilities = tf.nn.softmax(logits, axis=-1)
    #log_probs = tf.nn.log_softmax(logits, axis=-1)
    logits = tf.squeeze(logits)

    loss = tf.losses.mean_squared_error(label_ids,logits)


In [20]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
loss_ = loss.eval()
sess.close()

In [21]:
loss_

0.46543816